In [7]:
import sys
from elasticsearch import Elasticsearch
import requests
import lxml.html

In [8]:

es = Elasticsearch()

In [25]:
body = {'query': {'bool': {'must': [{'range': {'len': {'gte': 35}}},
    {'match': {'eng': {'query': 'big bear1',
       'fuzziness': '0',
       'minimum_should_match': 2}}}]}},
 'from': 0,
 'size': 10,
 'sort': ['_score', {'len': {'order': 'asc'}}]}

In [17]:
def filter_sentences(res):
    return True

In [18]:
def get_xpath(num):
    return '/html/body/div[3]/div[3]/div[5]/div[1]/ol[1]/li[%d]/a/text()' % num


def get_translate(word: str):
    api = requests.get('https://ru.wiktionary.org/wiki/' + word)
    tree = lxml.html.document_fromstring(api.text)
    pos_num = 1
    text_original = tree.xpath(get_xpath(pos_num))
    res_trans = []
    while len(text_original) > 0 and pos_num < 4:
        res_trans.append(text_original[0])
        pos_num += 1
        text_original = tree.xpath(get_xpath(pos_num))

    return res_trans

In [19]:
def make_es_query(query):
    q_len = len(query.split(' '))
    body['query']['bool']['must'][1]['match']['eng']['query']=query
    body['query']['bool']['must'][1]['match']['eng']['minimum_should_match']=q_len
    res = es.search(index="words25", body=body)
    ret_List = []
    #print (res)
    for res1 in res['hits']['hits']:
        #print (res1)
        if filter_sentences(res1):
            ret_List.append(res1)
    
    return ret_List

In [20]:
def process_es_res(res):
    if res:
        print (res[0]['_source']['eng'])
        print (res[0]['_source']['rus'])


In [21]:
def list_words_from_file(file):
    word_list = []
    with open(file) as f:
        for line in f:
            word_list.append(line.strip('\n'))
    return word_list

In [28]:
list_words = list_words_from_file('/home/igor/english/words/daily2.txt')

In [29]:
print ('#####')
for i,word in enumerate(list_words):
    print (word)
    trans = get_translate(word)
    print (", ".join(trans))
    # find 2 words
    w2_found = False
    for ii in range(1, len(list_words)-2):
        query = f'{word} {list_words[i-ii]}'
        res = make_es_query(query)
        #print (query)
        if res:
            process_es_res(res)
            w2_found = True
    if not w2_found:
        query = word
        res = make_es_query(query)
        process_es_res(res)
    print ('#####')  
        
        

#####
sophisticate
усложнять
#####
confident
уверенный
He's smart, confident, really hard.
Он умный, надежный, и очень горячий.
#####
unforseen

#####
whreabout

#####
appeal
воззвание, просьба, привлекательность
You told me you could arrange an appeal bond.
Вы гoвopили, чтo вытащите егo дo суда.
#####
siloed

#####
annoy
раздражать
Does it annoy you to find out that you're wrong about something?
Вас раздражает, когда вы ошибаетесь?
#####
tenacity
цепкость, упорство, вязкость
You need precision, tenacity, dedication.
Здесь нужна аккуратность, твердость воли, посвящение..
#####
disheart

#####
find out
понять, узнавать, вычитывать
Does it annoy you to find out that you're wrong about something?
Вас раздражает, когда вы ошибаетесь?
I suppose we'd better find out if the report is accurate, or merely rumor.
Я считаю, что нам нужно выяснить, правдивы наши донесения или это просто слухи.
Now, while we're here, why don't you and I find out how they're going to steal it and why?
А сейчас, пока

In [33]:
res = make_es_query("whreabout")
res

[]

In [31]:
import string
import re
from nltk.corpus import stopwords
import nltk
from collections import Counter
import operator
from nltk.corpus import words

In [32]:
stop_words = set(stopwords.words("english"))
punctuation_chars = string.punctuation.replace(",", "").replace("'", "")
english_words = set(words.words())

In [33]:
def remove_punct(text):  
    translator = str.maketrans(punctuation_chars, "."*len(punctuation_chars))
    return text.translate(translator)

In [40]:
def filter_string(txt,v=False):
    """return False - bad string"""
    
    txt = remove_punct(txt.lower())
    if txt.count('.')>1:
        return False, 'txt.count(\'.\')>1:'
    # filter numbers
    if bool(re.search(r'\d', txt)):
        return False, 'numbers'
    # 
    txt = txt.replace('.','').replace(',','')
    # -
    txt_words = nltk.word_tokenize(txt)
    if '.' in txt_words:
        txt_words.remove('.')
    if v: print (txt_words)
    without_stop_words = [word for word in txt_words if not word in stop_words]
    without_stop_words = [word for word in without_stop_words if '\''not in word]
    if v: print (without_stop_words)
    if len(without_stop_words)<3:
        return False , 'len(without_stop_words)<3'
    if len(without_stop_words)>10:
        return False,'len(without_stop_words)>10'
    # -
    without_not_eng_words = [word for word in without_stop_words if word in english_words]
    if v: print (without_not_eng_words)
    if len(without_stop_words)!=len(without_not_eng_words):
        return False, "without_not_eng_words"
    # -
    counts = Counter(without_not_eng_words)
    if counts.most_common(1)[0][1]>1:
        return False, 'counts.most_common>1'
    
    
    return True, ''
    

In [41]:
import re
def double_check(string,substr):
    return len([i for i in range(len(string)) if string.startswith(substr, i)])
def filter_sentences(res):
    #print (res)    
    str2 = res['_source']['eng']
    if  filter_string(str2)[0]==False:
       return False
    str2 = res['_source']['rus']
    if  double_check(str2,".")>1:
       return False
    return True
    

In [26]:
res = make_es_query("weird")

In [27]:
res
    

[{'_index': 'words25',
  '_type': '_doc',
  '_id': '1070953',
  '_score': 9.330822,
  '_source': {'eng': '"Some weird stuff, something secret',
   'rus': '"Прикольное место, секретная штука, ..."',
   'len': 35},
  'sort': [9.330822, 35]},
 {'_index': 'words25',
  '_type': '_doc',
  '_id': '1331491',
  '_score': 9.330822,
  '_source': {'eng': "You're probably feeling very weird.",
   'rus': '- Да. Твои ощущения очень странные.',
   'len': 35},
  'sort': [9.330822, 35]},
 {'_index': 'words25',
  '_type': '_doc',
  '_id': '1814344',
  '_score': 9.330822,
  '_source': {'eng': 'Something really weird is happening.',
   'rus': 'Происходит что-то действительно странное.',
   'len': 36},
  'sort': [9.330822, 36]},
 {'_index': 'words25',
  '_type': '_doc',
  '_id': '433364',
  '_score': 8.869225,
  '_source': {'eng': "Anyway, that's where it gets weird.",
   'rus': 'Короче, тут начинается самое странное.',
   'len': 35},
  'sort': [8.869225, 35]},
 {'_index': 'words25',
  '_type': '_doc',
  '_

In [194]:
x = [1,2,3,4,5]
x[-2]

4

In [397]:
res = make_es_query("regard")


UnboundLocalError: local variable '_str' referenced before assignment

In [36]:
nltk.word_tokenize("wqdqwd, qwfwqf.")

['wqdqwd', ',', 'qwfwqf', '.']

In [393]:
res = es.search(index="wwwords", body=body)

In [5]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /home/igor/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [26]:
nltk.download('punkt')
nltk.download('brown')
nltk.download('words')

[nltk_data] Downloading package punkt to /home/igor/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package brown to /home/igor/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package words to /home/igor/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [6]:
from nltk.corpus import stopwords

In [11]:
stopwords.words("english")[:2]

['i', 'me']

In [15]:
stop_words = set(stopwords.words("english"))
sentence = "Backgammon - is one of the oldest known board games."

words = nltk.word_tokenize(sentence)
without_stop_words = [word for word in words if not word in stop_words]
print(without_stop_words)

['Backgammon', '-', 'one', 'oldest', 'known', 'board', 'games', '.']


In [16]:
from nltk.corpus import brown

In [21]:
brown.words(categories='news')

['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', ...]

In [22]:
for sent in brown.sents():
    print(" ".join(sent))
    break

The Fulton County Grand Jury said Friday an investigation of Atlanta's recent primary election produced `` no evidence '' that any irregularities took place .


In [24]:
from nltk.corpus import words

In [26]:
"george" in words.words()

False

In [78]:
type(cv)

list

In [31]:
len(words.words())

236736

In [32]:
from nltk.corpus import brown



In [42]:
#!pip install mosestokenizer

In [49]:
#!pip install -U sacremoses

In [61]:
import googletrans
from googletrans import Translator

'3.0.0'